In [44]:
import os
import csv
import pandas as pd
import sqlite3

In [45]:
database_folder = "C:/Users/wayne/OneDrive/Desktop/CHS DVD Reader/East_West_DVD_Jul24"

In [46]:
current_database_path = "C:/Users/wayne/OneDrive/Desktop/CHS DVD Reader/East_West_DVD_Jul24/current_database.db"

In [47]:
def get_database_connection(database_path):
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()
    return conn, cursor

In [48]:
current_database_conn, current_database_cursor = get_database_connection(current_database_path)

In [49]:
current_database_cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables_current = [row[0] for row in current_database_cursor.fetchall()]

In [50]:
tables_current

['EastDVD_20240726_RM_ARC',
 'EastDVD_20240726_RM_ATL',
 'EastDVD_20240726_RM_CEN',
 'EastDVD_20240726_V_ATL_A',
 'EastDVD_20240726_V_ATL_B',
 'EastDVD_20240726_V_ATL_C',
 'EastDVD_20240726_V_CEN_A',
 'EastDVD_20240726_V_CEN_B',
 'EastDVD_20240726_V_CEN_C',
 'EastDVD_20240726_V_NOR_A',
 'WestDVD_20240726_RM_ARC',
 'WestDVD_20240726_RM_PAC',
 'WestDVD_20240726_V_NOR_A',
 'WestDVD_20240726_V_PAC_A',
 'WestDVD_20240726_V_PAC_B']

In [51]:
# Get the list of foldernames in the subject folder
folders = [item for item in os.listdir(database_folder) if os.path.isdir(os.path.join(database_folder, item))]

In [52]:
folders

['EastDVD_20240726', 'WestDVD_20240726']

In [70]:
raster_target_folder = 'BSBCHART'
vector_target_folder = 'ENC_ROOT'

In [68]:
def find_folder(starting_directory, target_folder_name):
    # Recursively searches for a folder with a specific name starting from the given directory.
    for root, dirs, files in os.walk(starting_directory):
        for dir_name in dirs:
            if dir_name == target_folder_name:
                return os.path.join(root, dir_name)

In [79]:
complete_path = []
for folder in folders:
    folder_path = os.path.join(database_folder, folder)
    # Get the list of sub-foldernames in the subject folder
    sub_folders = [item for item in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, item))]
    for sub_folder in sub_folders:
        sub_folder_path = os.path.join(folder_path, sub_folder)

        if "RM" in sub_folder_path:
            temp_path = find_folder(sub_folder_path, raster_target_folder)
        else:
            temp_path = find_folder(sub_folder_path, vector_target_folder)
    
        complete_path.append(os.path.dirname(temp_path))
    
print(complete_path)

['C:/Users/wayne/OneDrive/Desktop/CHS DVD Reader/East_West_DVD_Jul24\\EastDVD_20240726\\RM-ARC', 'C:/Users/wayne/OneDrive/Desktop/CHS DVD Reader/East_West_DVD_Jul24\\EastDVD_20240726\\RM-ATL', 'C:/Users/wayne/OneDrive/Desktop/CHS DVD Reader/East_West_DVD_Jul24\\EastDVD_20240726\\RM-CEN', 'C:/Users/wayne/OneDrive/Desktop/CHS DVD Reader/East_West_DVD_Jul24\\EastDVD_20240726\\V-ATL-A', 'C:/Users/wayne/OneDrive/Desktop/CHS DVD Reader/East_West_DVD_Jul24\\EastDVD_20240726\\V-ATL-B', 'C:/Users/wayne/OneDrive/Desktop/CHS DVD Reader/East_West_DVD_Jul24\\EastDVD_20240726\\V-ATL-C', 'C:/Users/wayne/OneDrive/Desktop/CHS DVD Reader/East_West_DVD_Jul24\\EastDVD_20240726\\V-CEN-A', 'C:/Users/wayne/OneDrive/Desktop/CHS DVD Reader/East_West_DVD_Jul24\\EastDVD_20240726\\V-CEN-B', 'C:/Users/wayne/OneDrive/Desktop/CHS DVD Reader/East_West_DVD_Jul24\\EastDVD_20240726\\V-CEN-C', 'C:/Users/wayne/OneDrive/Desktop/CHS DVD Reader/East_West_DVD_Jul24\\EastDVD_20240726\\V-NOR-A', 'C:/Users/wayne/OneDrive/Desktop

In [106]:
def find_and_read_csv(base_path):
    # Iterate through the directory structure
    for root, dirs, files in os.walk(base_path):
        # Check if 'RM' is in the directory path
        for file in files:
            # Check if the file is a .csv file
            if file.endswith('.csv'):
                csv_path = os.path.join(root, file)
                
                # Read the CSV file using pandas
                df = read_csv_with_fallback(csv_path)
                return df
    return None

In [107]:
def read_csv_with_fallback(file_path):
    """Try reading a CSV file with different encodings."""
    encodings = ['utf-8', 'latin1']
    for encoding in encodings:
        try:
            # note here I'm only reading the second column
            return pd.read_csv(file_path, usecols=[1], encoding=encoding, skiprows=1, header=None)
        except UnicodeDecodeError:
            continue
    raise ValueError(f"Could not read the file {file_path} with available encodings.")

In [ ]:
def convert_path_to_name(path):
    # Split the path into components
    components = path.split('\\')  # Split by backslash for Windows paths

    # Extract the relevant components
    parent_folder = components[-2]  # Second to last component
    last_folder = components[-1]  # Last component

    # Replace dashes and backslashes with underscores in the last folder name
    last_folder_cleaned = last_folder.replace('-', '_').replace('\\', '_')

    # Construct the final name
    final_name = f"{parent_folder}_{last_folder_cleaned}"
    
    return final_name

In [118]:
for item in complete_path:
    # Find and read the CSV file
    df = find_and_read_csv(item)
    
    formatted_name = convert_path_to_name(path)
  
    print(final_name)  # Output: EastDVD_20240726_RM_ARC
    
    try:
        # read the extracted second column of the CSV file to extract the list of expected_files
        expected_files = df.iloc[:, 0].tolist()





    
    #print(expected_files)

    # # Check for missing and extra files
    # missing = [file for file in expected_files if file not in content]
    # extra = [file for file in content if file not in expected_files]
    
    # if missing:
    #     missing_files.append(f"Missing files in {sub_sub_folder}: {missing}")
    # if extra:
    #     extra_files.appen(f"Extra files in {sub_sub_folder}: {extra}")
    except ValueError as e:
        print(f"Error reading CSV file {csv_file}: {e}")

EastDVD_20240726_RM_ARC
EastDVD_20240726_RM_ATL
EastDVD_20240726_RM_CEN
EastDVD_20240726_V_ATL_A
EastDVD_20240726_V_ATL_B
EastDVD_20240726_V_ATL_C
EastDVD_20240726_V_CEN_A
EastDVD_20240726_V_CEN_B
EastDVD_20240726_V_CEN_C
EastDVD_20240726_V_NOR_A
WestDVD_20240726_RM_ARC
WestDVD_20240726_RM_PAC
WestDVD_20240726_V_NOR_A
WestDVD_20240726_V_PAC_A
WestDVD_20240726_V_PAC_B
